In [1]:
import boto3
import time

In [2]:
def iterate_bucket(bucket):
    client = boto3.client('s3')
    paginator = client.get_paginator('list_objects_v2')
    page_iterator = paginator.paginate(Bucket=bucket)
    
    for page in page_iterator:
        if page['KeyCount'] > 0:
            for item in page['Contents']:
                yield item

In [5]:
pathlist=[]
start = time.time()
#for i in iterate_bucket(bucket='millionsongproject'):
for i in iterate_bucket(bucket='millionsongproject'):
    if i['Key'].endswith('.h5') and not i['Key']=='millionsongsubset/MillionSongSubset/AdditionalFiles/subset_msd_summary_file.h5':
        pathlist.append(i['Key'])
print("time passed = ", time.time()-start)

time passed =  194.2010359764099


In [6]:
import random
start = time.time()
random.shuffle(pathlist)
print("time passed = ", time.time()-start)

time passed =  0.7620112895965576


In [7]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql import SparkSession
sc = SparkContext()
ss=SparkSession \
.builder \
.getOrCreate()

In [8]:
path_rdd=sc.parallelize(pathlist)

In [10]:
start = time.time()
print(path_rdd.count())
print("time passed = ", time.time()-start)

1000000
time passed =  0.3231816291809082


In [11]:
import pandas as pd
import numpy as np

In [12]:
path_iter=path_rdd.toLocalIterator()

In [13]:
next(path_iter)

'L/M/I/TRLMICH128F9324111.h5'

In [14]:
analysis_sample_rate=[]
artist_7digitalid=[]
artist_familiarity=[]
artist_hotttnesss=[]
artist_id=[]
artist_latitude=[]
artist_location=[]
artist_longitude=[]
artist_mbid=[]
artist_mbtags=[]
artist_mbtags_count=[]
artist_name=[]
artist_playmeid=[]
artist_terms=[]
artist_terms_freq=[]
artist_terms_weight=[]
audio_md5=[]
bars_confidence=[]
bars_start=[]
beats_confidence=[]
beats_start=[]
danceability=[]
duration=[]
end_of_fade_in=[]
energy=[]
key=[]
key_confidence=[]
loudness=[]
mode=[]
mode_confidence=[]
release=[]
release_7digitalid=[]
sections_confidence=[]
sections_start=[]
segments_confidence=[]
segments_loudness_max=[]
segments_loudness_max_time=[]
segments_loudness_start=[]
segments_pitches=[]
segments_start=[]
segments_timbre=[]
similar_artists=[]
song_hotttnesss=[]
song_id=[]
start_of_fade_out=[]
tatums_confidence=[]
tatums_start=[]
tempo=[]
time_signature=[]
time_signature_confidence=[]
title=[]
track_7digitalid=[]
track_id=[]
year=[]

In [ ]:
import h5py
import s3fs
import h5py_getter
s3 = s3fs.S3FileSystem()
BUCKET='s3://millionsongproject/'
counter=0
start = time.time()
for a in path_iter:
    if counter < 200000:
        address=BUCKET+a
        with h5py.File(s3.open(address, 'rb'), 'r', lib_version='latest') as f:
            analysis_sample_rate.append(h5py_getter.get_analysis_sample_rate(f))
            #artist_7digitalid.append(h5py_getter.get_artist_7digitalid(f))
            artist_familiarity.append(h5py_getter.get_artist_familiarity(f))
            artist_hotttnesss.append(h5py_getter.get_artist_hotttnesss(f))
            artist_id.append(h5py_getter.get_artist_id(f))
            artist_latitude.append(h5py_getter.get_artist_latitude(f))
            artist_location.append(h5py_getter.get_artist_location(f))
            artist_longitude.append(h5py_getter.get_artist_longitude(f))
            #artist_mbid.append(h5py_getter.get_artist_mbid(f))
            #artist_mbtags.append(h5py_getter.get_artist_mbtags(f))
            #artist_mbtags_count.append(h5py_getter.get_artist_mbtags_count(f))
            artist_name.append(h5py_getter.get_artist_name(f))
            #artist_playmeid.append(h5py_getter.get_artist_playmeid(f))
            artist_terms.append(h5py_getter.get_artist_terms(f))
            artist_terms_freq.append(h5py_getter.get_artist_terms_freq(f))
            #artist_terms_weight.append(h5py_getter.get_artist_terms_weight(f))
            #audio_md5.append(h5py_getter.get_audio_md5(f))
            bars_confidence.append(h5py_getter.get_bars_confidence(f))
            bars_start.append(h5py_getter.get_bars_start(f))
            beats_confidence.append(h5py_getter.get_beats_confidence(f))
            beats_start.append(h5py_getter.get_beats_start(f))
            danceability.append(h5py_getter.get_danceability(f))
            duration.append(h5py_getter.get_duration(f))
            end_of_fade_in.append(h5py_getter.get_end_of_fade_in(f))
            energy.append(h5py_getter.get_energy(f))
            key.append(h5py_getter.get_key(f))
            key_confidence.append(h5py_getter.get_key_confidence(f))
            loudness.append(h5py_getter.get_loudness(f))
            mode.append(h5py_getter.get_mode(f))
            mode_confidence.append(h5py_getter.get_mode_confidence(f))
            release.append(h5py_getter.get_release(f))
            #release_7digitalid.append(h5py_getter.get_release_7digitalid(f))
            sections_confidence.append(h5py_getter.get_sections_confidence(f))
            sections_start.append(h5py_getter.get_sections_start(f))
            segments_confidence.append(h5py_getter.get_segments_confidence(f))
            segments_loudness_max.append(h5py_getter.get_segments_loudness_max(f))
            segments_loudness_max_time.append(h5py_getter.get_segments_loudness_max_time(f))
            segments_loudness_start.append(h5py_getter.get_segments_loudness_start(f))
            segments_pitches.append(h5py_getter.get_segments_pitches(f))
            segments_start.append(h5py_getter.get_segments_start(f))
            segments_timbre.append(h5py_getter.get_segments_timbre(f))
            similar_artists.append(h5py_getter.get_similar_artists(f))
            song_hotttnesss.append(h5py_getter.get_song_hotttnesss(f))
            song_id.append(h5py_getter.get_song_id(f))
            start_of_fade_out.append(h5py_getter.get_start_of_fade_out(f))
            tatums_confidence.append(h5py_getter.get_tatums_confidence(f))
            tatums_start.append(h5py_getter.get_tatums_start(f))
            tempo.append(h5py_getter.get_tempo(f))
            time_signature.append(h5py_getter.get_time_signature(f))
            time_signature_confidence.append(h5py_getter.get_time_signature_confidence(f))
            title.append(h5py_getter.get_title(f))
            #track_7digitalid.append(h5py_getter.get_track_7digitalid(f))
            #track_id.append(h5py_getter.get_track_id(f))
            year.append(h5py_getter.get_year(f))
            counter +=1
            print("\rNumber of files done = ", counter, end=' ')
    else:
        break
print("time passed = ", time.time()-start)    

Number of files done =  81 

/home/hadoop/miniconda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/hadoop/miniconda/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Number of files done =  25948 6  5331 10717 15067 

In [ ]:
len(title)

In [ ]:
song_id[:5]

In [ ]:
start = time.time()
df = pd.DataFrame()
df['analysis_sample_rate']=pd.Series(analysis_sample_rate)
#df['artist_7digitalid']=pd.Series(artist_7digitalid)
df['artist_familiarity']=pd.Series(artist_familiarity)
df['artist_hotttnesss']=pd.Series(artist_hotttnesss)
df['artist_id']=pd.Series([s.decode('utf-8') for s in artist_id])
df['artist_latitude']=pd.Series(artist_latitude)
df['artist_location']=pd.Series([s.decode('utf-8') for s in artist_location])
df['artist_longitude']=pd.Series(artist_longitude)
#df['artist_mbid']=pd.Series(artist_mbid)
#df['artist_mbtags']=pd.Series(artist_mbtags)
#df['artist_mbtags_count']=pd.Series(artist_mbtags_count)
df['artist_name']=pd.Series([s.decode('utf-8') for s in artist_name])
#df['artist_playmeid']=pd.Series(artist_playmeid)
df['artist_terms']=pd.Series(s.astype('U') for s in artist_terms)
df['artist_terms_freq']=pd.Series(artist_terms_freq)
#df['artist_terms_weight']=pd.Series(artist_terms_weight)
#df['audio_md5']=pd.Series(audio_md5)
df['bars_confidence']=pd.Series(bars_confidence)
df['bars_start']=pd.Series(bars_start)
df['beats_confidence']=pd.Series(beats_confidence)
df['beats_start']=pd.Series(beats_start)
df['danceability']=pd.Series(danceability)
df['duration']=pd.Series(duration)
df['end_of_fade_in']=pd.Series(end_of_fade_in)
df['energy']=pd.Series(energy)
df['key']=pd.Series(key)
df['key_confidence']=pd.Series(key_confidence)
df['loudness']=pd.Series(loudness)
df['mode']=pd.Series(mode)
df['mode_confidence']=pd.Series(mode_confidence)
df['release']=pd.Series([s.decode('utf-8') for s in release])
#df['release_7digitalid']=pd.Series(release_7digitalid)
df['sections_confidence']=pd.Series(sections_confidence)
df['sections_start']=pd.Series(sections_start)
df['segments_confidence']=pd.Series(segments_confidence)
df['segments_loudness_max']=pd.Series(segments_loudness_max)
df['segments_loudness_max_time']=pd.Series(segments_loudness_max_time)
df['segments_loudness_start']=pd.Series(segments_loudness_start)
df['segments_pitches']=pd.Series(segments_pitches)
df['segments_start']=pd.Series(segments_start)
df['segments_timbre']=pd.Series(segments_timbre)
df['similar_artists']=pd.Series(similar_artists)
df['song_hotttnesss']=pd.Series(song_hotttnesss)
df['song_id']=pd.Series([s.decode('utf-8') for s in song_id])
df['start_of_fade_out']=pd.Series(start_of_fade_out)
df['tatums_confidence']=pd.Series(tatums_confidence)
df['tatums_start']=pd.Series(tatums_start)
df['tempo']=pd.Series(tempo)
df['time_signature']=pd.Series(time_signature)
df['time_signature_confidence']=pd.Series(time_signature_confidence)
df['title']=pd.Series([s.decode('utf-8') for s in title])
#df['track_7digitalid']=pd.Series(track_7digitalid)
#df['track_id']=pd.Series(track_id)
df['year']=pd.Series(year)
print("time passed = ", time.time()-start)

In [ ]:
df.head(5)

In [ ]:
df.to_csv('big_subset.csv', index=False)

In [ ]:
test_file = ss.read.csv('big_subset.csv', header=True)

In [ ]:
test_file.schema

In [ ]:
test_file.show(10)

In [ ]:
a=test_file.take(1)

In [ ]:
a

In [ ]:
def append_rdd(feature_rdd, data):
    data=data.tolist()
    data=[data]
    temp_rdd=sc.parallelize(data)
    return feature_rdd.union(temp_rdd)

In [ ]:
import h5py
import s3fs
import h5py_getter
from pyspark.sql import Row
s3 = s3fs.S3FileSystem()
address='s3://cosc502millionsong/millionsongsubset/MillionSongSubset/data/A/A/A/TRAAAAW128F429D538.h5'
attributes = ['artist_7digitalid','artist_familiarity','artist_hotttnesss','artist_id',
     'artist_latitude','artist_location','artist_longitude','artist_mbidartist_mbtags',
     'artist_mbtags_count','artist_name','artist_playmeid','artist_terms',
     'artist_terms_freq','artist_terms_weight','audio_md5', 'bars_confidence','bars_start',
     'beats_confidence','beats_start','danceability','duration','end_of_fade_in',
     'energy','key','key_confidence','loudness','mode','mode_confidence',
     'release','release_7digitalid','sections_confidence','sections_start',
     'segments_confidence','segments_loudness_max','segments_loudness_max_time',
     'segments_loudness_start','segments_pitches','segments_start','segments_timbre',
     'similar_artists','song_hotttnesss','song_id','start_of_fade_out','tatums_confidence',
     'tatums_start','tempo','time_signature','time_signature_confidence','title','track_7digitalid',
     'track_id','year']
with h5py.File(s3.open(address, 'rb'), 'r', lib_version='latest') as f:
    analysis_sample_rate=append_rdd(analysis_sample_rate, h5py_getter.get_analysis_sample_rate(f))
    artist_7digitalid=append_rdd(artist_7digitalid, h5py_getter.get_artist_7digitalid(f))
    artist_familiarity=append_rdd(artist_familiarity, h5py_getter.get_artist_familiarity(f))
    artist_hotttnesss=append_rdd(artist_hotttnesss, h5py_getter.get_artist_hotttnesss(f))
    artist_id=append_rdd(artist_id, h5py_getter.get_artist_id(f))
    artist_latitude=append_rdd(artist_latitude, h5py_getter.get_artist_latitude(f))
    artist_location=append_rdd(artist_location, h5py_getter.get_artist_location(f))
    artist_longitude=append_rdd(artist_longitude, h5py_getter.get_artist_longitude(f))
    artist_mbid=append_rdd(artist_mbid, h5py_getter.get_artist_mbid(f))
    artist_mbtags=append_rdd(artist_mbtags, h5py_getter.get_artist_mbtags(f))
    artist_mbtags_count=append_rdd(artist_mbtags_count, h5py_getter.get_artist_mbtags_count(f))
    artist_name=append_rdd(artist_name, h5py_getter.get_artist_name(f))
    artist_playmeid=append_rdd(artist_playmeid, h5py_getter.get_artist_playmeid(f))
    artist_terms=append_rdd(artist_terms, h5py_getter.get_artist_terms(f))
    artist_terms_freq=append_rdd(artist_terms_freq, h5py_getter.get_artist_terms_freq(f))
    artist_terms_weight=append_rdd(artist_terms_weight, h5py_getter.get_artist_terms_weight(f))
    audio_md5=append_rdd(audio_md5, h5py_getter.get_audio_md5(f))
    bars_confidence=append_rdd(bars_confidence, h5py_getter.get_bars_confidence(f))
    bars_start=append_rdd(bars_start, h5py_getter.get_bars_start(f))
    beats_confidence=append_rdd(beats_confidence, h5py_getter.get_beats_confidence(f))
    beats_start=append_rdd(beats_start, h5py_getter.get_beats_start(f))
    danceability=append_rdd(danceability, h5py_getter.get_danceability(f))
    duration=append_rdd(duration, h5py_getter.get_duration(f))
    end_of_fade_in=append_rdd(end_of_fade_in, h5py_getter.get_end_of_fade_in(f))
    energy=append_rdd(energy, h5py_getter.get_energy(f))
    key=append_rdd(key, h5py_getter.get_key(f))
    key_confidence=append_rdd(key_confidence, h5py_getter.get_key_confidence(f))
    loudness=append_rdd(loudness, h5py_getter.get_loudness(f))
    mode=append_rdd(mode, h5py_getter.get_mode(f))
    mode_confidence=append_rdd(mode_confidence, h5py_getter.get_mode_confidence(f))
    release=append_rdd(release, h5py_getter.get_release(f))
    release_7digitalid=append_rdd(release_7digitalid, h5py_getter.get_release_7digitalid(f))
    sections_confidence=append_rdd(sections_confidence, h5py_getter.get_sections_confidence(f))
    sections_start=append_rdd(sections_start, h5py_getter.get_sections_start(f))
    segments_confidence=append_rdd(segments_confidence, h5py_getter.get_segments_confidence(f))
    segments_loudness_max=append_rdd(segments_loudness_max, h5py_getter.get_segments_loudness_max(f))
    segments_loudness_max_time=append_rdd(segments_loudness_max_time, h5py_getter.get_segments_loudness_max_time(f))
    segments_loudness_start=append_rdd(segments_loudness_start, h5py_getter.get_segments_loudness_start(f))
    segments_pitches=append_rdd(segments_pitches, h5py_getter.get_segments_pitches(f))
    segments_start=append_rdd(segments_start, h5py_getter.get_segments_start(f))
    segments_timbre=append_rdd(segments_timbre, h5py_getter.get_segments_timbre(f))
    similar_artists=append_rdd(similar_artists, h5py_getter.get_similar_artists(f))
    song_hotttnesss=append_rdd(song_hotttnesss, h5py_getter.get_song_hotttnesss(f))
    song_id=append_rdd(song_id, h5py_getter.get_song_id(f))
    start_of_fade_out=append_rdd(start_of_fade_out, h5py_getter.get_start_of_fade_out(f))
    tatums_confidence=append_rdd(tatums_confidence, h5py_getter.get_tatums_confidence(f))
    tatums_start=append_rdd(tatums_start, h5py_getter.get_tatums_start(f))
    tempo=append_rdd(tempo, h5py_getter.get_tempo(f))
    time_signature=append_rdd(time_signature, h5py_getter.get_time_signature(f))
    time_signature_confidence=append_rdd(time_signature_confidence, h5py_getter.get_time_signature_confidence(f))
    title=append_rdd(title, h5py_getter.get_title(f))
    track_7digitalid=append_rdd(track_7digitalid, h5py_getter.get_track_7digitalid(f))
    track_id=append_rdd(track_id, h5py_getter.get_track_id(f))
    year=append_rdd(year, h5py_getter.get_year(f))
errors=0

row=Row('analysis_sample_rate')
df = analysis_sample_rate.map(row).toDF()
for attribute in attributes:
    row=Row(attribute)
    tempdf = globals()[attribute].map(row).toDF()
    #df=df.withColumn(attribute, lit(None)).select(df.columns)
    #for c in df.columns:
        #tempdf=tempdf.withColumn(c, lit(None)).select(tempdf.columns)
    df=df.union(tempdf)

   

